# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [2]:
import findspark
findspark.init('/home/elias/spark-3.0.0-preview2-bin-hadoop2.7')
import pyspark

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Churn').getOrCreate()

In [18]:
df = spark.read.csv('customer_churn.csv' , inferSchema= True, header=True )

In [19]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [38]:
from pyspark.sql.functions import format_number,dayofmonth,hour,dayofyear,month,year,weekofyear,date_format
date_test = df.withColumn("Onboard_date" , year(df["Onboard_date"]))
                                           
#,"MM/dd/yy hh:mm"))
#select(unix ['Onboard_date'].

In [33]:
Date.show()

+-------------------+----+--------------+---------------+-----+---------+------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|        2013|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|        2013|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|        2016|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|        2014|13120 Daniel Moun...|           Smith Inc|    1|
|     Cynthia Norton|37.0|       9191.58|              

In [34]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [39]:

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(
    inputCols=["Age", 
               "Total_Purchase",'Account_Manager','Num_Sites'],
    outputCol="features")

Assem_DF = assembler.transform(df)

In [40]:
Final_DF= Assem_DF.select("features","churn")

In [41]:
train_df , test_df = Final_DF.randomSplit([0.7,0.3])

In [43]:
#Build the linear regression model
from pyspark.ml.classification import LogisticRegression
LR = LogisticRegression(labelCol='churn',featuresCol="features")
LR_model= LR.fit(train_df)

In [94]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator
results = LR_model.transform(train_df)
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn'  )
AUC = my_eval.evaluate(results)
AUC

0.7148148148148149

In [97]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(LR.regParam, [0.01, 0.5, 2.0])
             .addGrid(LR.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(LR.maxIter, [1, 5, 10])
             .build())

In [98]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=LR, estimatorParamMaps=paramGrid, evaluator=my_eval, numFolds=5)

# Run cross validations
cvModel = cv.fit(train_df)
# this will likely take a fair amount of time because of the amount of models that we're creating and testing

In [99]:
predictions = cvModel.transform(test_df)
my_eval.evaluate(predictions)

0.6015873015873016

In [100]:
print('Model Intercept: ', cvModel.bestModel.intercept)

Model Intercept:  -3.0770142952492776


In [115]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pyspark.sql 
  
sqlContext = SQLContext(spark)
weights = cvModel.bestModel.coefficients
weights = [(float(w),) for w in weights]  # convert numpy type to float, and to tuple
weightsDF = sqlContext.createDataFrame(weights, ["Feature Weight"])
weightsDF.show()

best_parameters = cvModel.bestModel.getRegParam

print(cvModel.bestModel.getRegParam())
print(cvModel.bestModel.getElasticNetParam())
print(cvModel.bestModel.getMaxIter())

 

+--------------------+
|      Feature Weight|
+--------------------+
| -0.0915852870866686|
|-1.64452212694323...|
| 0.09743114546689409|
|  0.7282433528499027|
+--------------------+

0.01
0.0
10
